<a href="https://colab.research.google.com/github/douglascsantos/colab_analyse_text/blob/main/api_pubmed_scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
try:
  import google.colab
  !pip install gspread
  !pip install gspread-dataframe
  !pip install gspread-formatting
  !pip install --upgrade pandas --user
  !pip install ipynb
  !pip install biopython
  !pip install pymed
except ImportError:
    pass

In [8]:
import os
import sys
from threading import Thread
import time
import xml.etree.ElementTree as et
import pandas as pd

from urllib.request import urlretrieve

import Bio

from Bio import SeqIO, SearchIO, Entrez
#from Bio.Seq import Seq
#from Bio.SeqUtils import GC
#from Bio.Blast import NCBIWWW
#from Bio.Data import CodonTable

from google.colab import auth
import gspread
from oauth2client.client import GoogleCredentials
import gspread_dataframe
from gspread_dataframe import get_as_dataframe, set_with_dataframe
import gspread_formatting
from gspread_formatting.dataframe import format_with_dataframe



Entrez.email = "douglas.santos.mat@gmail.com"
Entrez.api_key = "ce7112eb9027e9ffe673c980fea463c08c08"

print("Python version:", sys.version_info)
print("Biopython version:", Bio.__version__)

ModuleNotFoundError: ignored

In [ ]:
#SESSÃO DE VARIAVEIS E DECLARAÇÕES
palavra_chave_da_busca = 'cancer'#'NM_004333.4 c.2128-27C>T'
qtde_artigos_retorno = 20
retorno_acesso_api_pubmed = '' #retorno do acesso ao pubmed
numero_de_artigos_disponiveis = '' 
xml_retorno = ''
lista_de_PMID = []
split_lista_de_PMID = []
lista_info_artigo = ['ForeName', 'LastName', 'Initials', 'Affiliation', \
                    'Title', 'PMID', 'ISSNLinking','ELocationID', 'ArticleId', 'Year', 'Month', 'Day', \
                    'ArticleTitle', 'AbstractText', 'Language', 'Keyword']
lista_PMID_ERRO = []
dic_info_artigo = {}
dict_informacoes_recebidas = {}
COUNTDOWN = 1
pmid_num = ''
#sql_comando = '' #criar, armazenar_dadosbuscar, deletar, update_dados
df_planilha = pd.DataFrame(columns=lista_info_artigo)
#print(df)
####FIM DA SESSÃO DE VARIAVEIS E DECLARAÇÕES




In [ ]:
### FUNÇÕES PARA API PUBMED ##########################
def get_abstract_print(pmid):
    try:
        handle = Entrez.efetch(db='pubmed', id=pmid, retmode='text', rettype='abstract')
        abstract_txt = handle.read()
        #print(x)
        return abstract_txt
        
    except InterruptedError:
        return None
                            

def get_abstract_info(pmid):
    try:
        handle = Entrez.efetch(db='pubmed', id=pmid, retmode='xml')
        xml_data = handle.read()
        return xml_data

    except InterruptedError:
        return None
    
def get_abstract(palavra_chave, qtde_artigos_retorno):
    try:
        handle = Entrez.esearch(db='pubmed', retmax=qtde_artigos_retorno, sort='relevance', term=palavra_chave)
        record = Entrez.read(handle)
        #first = record["IdList"][0]
        lista_de_pmids = record['IdList']
        numero_artigos_localizados = record.get('Count')
        #numero_artigos_lista = int(len(record['IdList']))
        return numero_artigos_localizados, lista_de_pmids

    except InterruptedError:
        return None

#print('numero de artigos encontrados: ' + str(len(record))) # this matches http://www.ncbi.nlm.nih.gov/pubmed/?term=pyramidal%20cell

def localiza_atributos(xml_data):
  lista_aux = []
  dict_aux = {}
  root = et.fromstring(xml_data)
  
  for colname in lista_info_artigo:
      element = root.iter(colname)
      #print('------------------------')
      #print("coluna nome: " + str(colname))

      for sub_element in element:

        lista_aux.append(sub_element.text)    
        
      dict_aux[colname] = str(lista_aux).replace("'", "")[1:-1]
      #df = df.append(pd.DataFrame(lista_aux, columns=[colname]))
      #print('------------------------------------')
      lista_aux.clear()
      
  return dict_aux

In [ ]:
######### INICIO DO PROGRAMA CLASSE THREAD SERVE PARA CHAMAR UMA NOVA INSTANCIA A CADA BUSCA ######################################
class Th(Thread):

    def __init__ (self, num): #, pmid_num):
        sys.stdout.write("Abrindo a thread numero -> " + str(num) + "\n")
        sys.stdout.flush()
        Thread.__init__(self)
        #self.pmid_num = pmid_num
        self.num = num
        self.countdown = COUNTDOWN
        time.sleep(1)
        #print('\n')

    def run(self):
      
      xml_retorno = get_abstract_info(self.num) 
      global df_planilha
      dicionario1 = localiza_atributos(xml_retorno)
      #print('-----------------------'+'\n'+' dicionario retorno: '+ str(dicionario1))
      
      df = pd.DataFrame([dicionario1], index=[0])
      df_planilha = df_planilha.append(df)
      df_planilha.reset_index(drop=True, inplace=True)     

      if xml_retorno == None: 
        print('erro ao tentar obter o xml_data de: ' + str(self.num))
      else: 
        exit(0)
   
       # print('------------------------------------------------------ ok ' + str(self.num))

In [ ]:

## O CODIGO COMEÇA AQUI  
if (palavra_chave_da_busca) and (int(len(palavra_chave_da_busca)) < 15):
    numero_de_artigos_disponiveis, lista_de_PMID = get_abstract(palavra_chave_da_busca, qtde_artigos_retorno)
    print('numero de artigos encontrados: ' + str(numero_de_artigos_disponiveis))
    print('numero de artigos selecionados: ' + str(qtde_artigos_retorno))
    print(lista_de_PMID)

    #tratamento de falha!
if (int(numero_de_artigos_disponiveis) == 0): # or (numero_de_artigos_disponiveis == None): 
    print('nada encontrado ou erro de acesso   =P')
    exit(0) 

for PMID in lista_de_PMID:
    pmid_artigo = PMID
    #print(pmid_artigo)
    thread_number = PMID
    thread = Th(thread_number)
    #thread.start()
    thread.run()
    print('thread: '+ str(PMID) + ' concluida com sucesso!')
    #print('resultado df:')
    #print(df_planilha.head(10))


In [ ]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

In [ ]:
#worksheet = gc.open('nome planilha').sheet1
planilha = gc.open('resumo artigos cientificos')
worksheet = planilha.get_worksheet(0)

In [ ]:


# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
print(rows)
dataframe = pd.DataFrame(worksheet.get_all_records())
dataframe.head()
#df_gsuite = pd.DataFrame.from_records(rows, columns = rows[0] )
#df_gsuite.drop(1, axis = 0, inplace = True)
#df_gsuite.drop(0, axis = 0, inplace = True)

In [ ]:
#worksheet.update([df_planilha.columns.values.tolist()] + df_planilha.values.tolist())
#worksheet.write([df_planilha.columns.values.tolist()] + df_planilha.values.tolist())

#worksheet = some_worksheet_obtained_from_gspread_client

#df = pd.DataFrame.from_records([{'a': i, 'b': i * 2} for i in range(100)])
set_with_dataframe(worksheet, df_planilha)
format_with_dataframe(worksheet, df_planilha, include_column_header=True)